# 03 - Revenue Forecasting Model

## Aurora Utensils Manufacturing (AUM)

This notebook builds and evaluates revenue forecasting models using:
- Monthly sales data (2016-2020)
- Engineered features (lags, rolling means, seasonality)
- Optional macro indicators (IPMAN, AMTMVS)

### Sections
1. Setup & Data Loading
2. Baseline Models (Naive, Seasonal, Moving Average)
3. ML Models (Random Forest, Gradient Boosting)
4. Final Model Selection
5. Forecast Next 12 Months

---

## 1. Setup & Data Loading

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')

# ML imports
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

# Project imports
import sys
sys.path.insert(0, '../src')
from data.load_data import load_config, load_macro_ip_data, load_macro_msi_data
from data.make_features import build_forecasting_features, split_time_series

# Plot settings
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11
sns.set_palette('husl')

# Ensure output directories exist
os.makedirs('../reports/figures', exist_ok=True)
os.makedirs('../data/interim', exist_ok=True)
os.makedirs('../models', exist_ok=True)

print("✓ Setup complete")
print(f"  Analysis date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# Load configuration and data
config = load_config()

# Load monthly sales
df_monthly = pd.read_csv('../data/interim/monthly_sales.csv')
df_monthly['year_month'] = pd.to_datetime(df_monthly['year_month'])
print(f"Monthly sales: {len(df_monthly)} rows")
print(f"Date range: {df_monthly['year_month'].min()} to {df_monthly['year_month'].max()}")

# Load macro data
try:
    df_ip = load_macro_ip_data(config)
    print(f"IPMAN loaded: {len(df_ip)} rows")
except FileNotFoundError:
    df_ip = None
    print("⚠ IPMAN not available")

try:
    df_msi = load_macro_msi_data(config)
    print(f"MSI loaded: {len(df_msi)} rows")
except FileNotFoundError:
    df_msi = None
    print("⚠ MSI not available")

df_monthly.head()

In [ ]:
# Build forecasting features
df_features = build_forecasting_features(df_monthly, df_ip=df_ip, df_msi=df_msi)
print(f"Features shape: {df_features.shape}")
print(f"\nFeature columns:")
print(df_features.columns.tolist())
df_features.head(15)

In [ ]:
# Split into train (2016-2018) and validation (2019-2020)
# Drop rows with NaN features (first 12 months due to lags)
df_clean = df_features.dropna().copy()
print(f"Clean data: {len(df_clean)} rows (after dropping NaN from lags)")

train_df, valid_df = split_time_series(df_clean, '2019-01-01')
print(f"\nTrain: {len(train_df)} months ({train_df['month'].min().strftime('%Y-%m')} to {train_df['month'].max().strftime('%Y-%m')})")
print(f"Valid: {len(valid_df)} months ({valid_df['month'].min().strftime('%Y-%m')} to {valid_df['month'].max().strftime('%Y-%m')})")

In [ ]:
# Define feature columns (excluding month and target)
FEATURE_COLS = [c for c in df_clean.columns if c not in ['month', 'revenue']]
TARGET_COL = 'revenue'

# Prepare X and y
X_train = train_df[FEATURE_COLS]
y_train = train_df[TARGET_COL]
X_valid = valid_df[FEATURE_COLS]
y_valid = valid_df[TARGET_COL]

print(f"Features: {len(FEATURE_COLS)}")
print(FEATURE_COLS)

---
## 2. Baseline Models

In [ ]:
# Helper functions for metrics
def calculate_metrics(y_true, y_pred):
    """Calculate MAE, RMSE, and MAPE."""
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    # MAPE with protection against division by zero
    mape = np.mean(np.abs((y_true - y_pred) / y_true.replace(0, np.nan))) * 100
    return {'MAE': mae, 'RMSE': rmse, 'MAPE': mape}

def display_metrics(metrics_dict, model_name):
    """Display metrics for a model."""
    print(f"\n{model_name}:")
    print(f"  MAE:  ₹{metrics_dict['MAE']:,.0f}")
    print(f"  RMSE: ₹{metrics_dict['RMSE']:,.0f}")
    print(f"  MAPE: {metrics_dict['MAPE']:.2f}%")

In [ ]:
# Store all model results
results = {}

# Baseline 1: Naive-1 (forecast = previous month's revenue)
# Using revenue_lag_1 feature
pred_naive1 = valid_df['revenue_lag_1'].values
results['Naive-1'] = calculate_metrics(y_valid, pred_naive1)
display_metrics(results['Naive-1'], 'Naive-1 (Previous Month)')

In [ ]:
# Baseline 2: Seasonal Naive-12 (forecast = same month last year)
# Using revenue_lag_12 feature
pred_naive12 = valid_df['revenue_lag_12'].values
results['Seasonal Naive-12'] = calculate_metrics(y_valid, pred_naive12)
display_metrics(results['Seasonal Naive-12'], 'Seasonal Naive-12 (Same Month Last Year)')

In [ ]:
# Baseline 3: Moving Average (last 3 months mean)
# Using revenue_roll_mean_3 feature
pred_ma3 = valid_df['revenue_roll_mean_3'].values
results['Moving Avg 3'] = calculate_metrics(y_valid, pred_ma3)
display_metrics(results['Moving Avg 3'], 'Moving Average (3 months)')

In [ ]:
# Baseline 4: Moving Average (last 6 months mean)
pred_ma6 = valid_df['revenue_roll_mean_6'].values
results['Moving Avg 6'] = calculate_metrics(y_valid, pred_ma6)
display_metrics(results['Moving Avg 6'], 'Moving Average (6 months)')

In [ ]:
# Summary of baseline models
print("\n" + "="*60)
print("BASELINE MODEL COMPARISON")
print("="*60)
baseline_df = pd.DataFrame(results).T
baseline_df = baseline_df.round(2)
display(baseline_df)

---
## 3. ML Models

In [ ]:
# Random Forest Regressor
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=3,
    min_samples_leaf=2,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)
pred_rf = rf_model.predict(X_valid)

results['Random Forest'] = calculate_metrics(y_valid, pred_rf)
display_metrics(results['Random Forest'], 'Random Forest')

In [ ]:
# Gradient Boosting Regressor
gb_model = GradientBoostingRegressor(
    n_estimators=100,
    max_depth=5,
    learning_rate=0.1,
    min_samples_split=3,
    min_samples_leaf=2,
    random_state=42
)

gb_model.fit(X_train, y_train)
pred_gb = gb_model.predict(X_valid)

results['Gradient Boosting'] = calculate_metrics(y_valid, pred_gb)
display_metrics(results['Gradient Boosting'], 'Gradient Boosting')

In [ ]:
# Feature importance from Random Forest
feature_importance = pd.DataFrame({
    'feature': FEATURE_COLS,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 8))
plt.barh(range(len(feature_importance)), feature_importance['importance'].values, color='steelblue')
plt.yticks(range(len(feature_importance)), feature_importance['feature'].values)
plt.xlabel('Importance')
plt.title('Random Forest Feature Importance')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('../reports/figures/feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 4. Final Model Selection

In [ ]:
# Compare all models
print("\n" + "="*60)
print("ALL MODEL COMPARISON")
print("="*60)

all_results_df = pd.DataFrame(results).T
all_results_df = all_results_df.round(2)
all_results_df = all_results_df.sort_values('MAPE')
display(all_results_df)

# Select best model based on MAPE
best_model_name = all_results_df.index[0]
print(f"\n🏆 Best model: {best_model_name} (MAPE: {all_results_df.loc[best_model_name, 'MAPE']:.2f}%)")

In [ ]:
# Determine which model and predictions to use
if best_model_name == 'Random Forest':
    best_model = rf_model
    best_pred = pred_rf
elif best_model_name == 'Gradient Boosting':
    best_model = gb_model
    best_pred = pred_gb
else:
    # If baseline is best, still use Gradient Boosting for forecasting
    print(f"Note: Baseline '{best_model_name}' is best, but using Gradient Boosting for future forecasts")
    best_model = gb_model
    best_pred = pred_gb
    best_model_name = 'Gradient Boosting'

In [ ]:
# Plot: Actual vs Predicted for validation period
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Time series plot
axes[0].plot(valid_df['month'], y_valid, 'o-', label='Actual', color='#2E86AB', linewidth=2, markersize=6)
axes[0].plot(valid_df['month'], best_pred, 's--', label='Predicted', color='#E94F37', linewidth=2, markersize=6)
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Revenue (₹)')
axes[0].set_title(f'Actual vs Predicted Revenue - Validation Period ({best_model_name})')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residuals plot
residuals = y_valid.values - best_pred
axes[1].bar(valid_df['month'], residuals, color=['#2E86AB' if r >= 0 else '#E94F37' for r in residuals], alpha=0.7)
axes[1].axhline(0, color='black', linewidth=1)
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Residual (Actual - Predicted)')
axes[1].set_title('Residuals Over Time')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/figures/forecast_validation.png', dpi=150, bbox_inches='tight')
plt.show()

---
## 5. Forecast Next 12 Months

In [ ]:
# Generate forecast for next 12 months (2021-01 to 2021-12)
# We need to roll forward, generating features for each month

def generate_future_features(df_history, n_months, df_ip=None, df_msi=None):
    """
    Generate features for future months by rolling forward.
    Uses predicted revenue as input for subsequent lags.
    """
    # Start with last available data
    last_date = df_history['month'].max()
    
    # Create copy of history for rolling
    history = df_history.copy()
    future_predictions = []
    
    for i in range(n_months):
        # Next month
        next_month = last_date + pd.DateOffset(months=i+1)
        
        # Create feature row for this month
        row = {
            'month': next_month,
            'year': next_month.year,
            'month_num': next_month.month,
            'quarter': (next_month.month - 1) // 3 + 1,
            'is_year_end': 1 if next_month.month == 12 else 0,
            'is_year_start': 1 if next_month.month == 1 else 0,
            'is_quarter_end': 1 if next_month.month in [3, 6, 9, 12] else 0,
            'month_sin': np.sin(2 * np.pi * next_month.month / 12),
            'month_cos': np.cos(2 * np.pi * next_month.month / 12),
        }
        
        # Get revenue from history (including previous predictions)
        all_revenue = list(history['revenue'].values) + [p['revenue_forecast'] for p in future_predictions]
        
        # Lag features
        row['revenue_lag_1'] = all_revenue[-1] if len(all_revenue) >= 1 else np.nan
        row['revenue_lag_3'] = all_revenue[-3] if len(all_revenue) >= 3 else np.nan
        row['revenue_lag_6'] = all_revenue[-6] if len(all_revenue) >= 6 else np.nan
        row['revenue_lag_12'] = all_revenue[-12] if len(all_revenue) >= 12 else np.nan
        
        # Rolling features (using shift to avoid leakage)
        if len(all_revenue) >= 4:
            row['revenue_roll_mean_3'] = np.mean(all_revenue[-4:-1])
        else:
            row['revenue_roll_mean_3'] = np.nan
            
        if len(all_revenue) >= 7:
            row['revenue_roll_mean_6'] = np.mean(all_revenue[-7:-1])
        else:
            row['revenue_roll_mean_6'] = np.nan
            
        if len(all_revenue) >= 13:
            row['revenue_roll_mean_12'] = np.mean(all_revenue[-13:-1])
        else:
            row['revenue_roll_mean_12'] = np.nan
            
        if len(all_revenue) >= 4:
            row['revenue_roll_std_3'] = np.std(all_revenue[-4:-1])
        else:
            row['revenue_roll_std_3'] = np.nan
            
        if len(all_revenue) >= 7:
            row['revenue_roll_std_6'] = np.std(all_revenue[-7:-1])
        else:
            row['revenue_roll_std_6'] = np.nan
        
        # Quantity lags (use average if not available)
        if 'quantity_lag_1' in FEATURE_COLS:
            avg_qty = history['total_quantity'].mean() if 'total_quantity' in history.columns else 0
            row['quantity_lag_1'] = avg_qty
            row['quantity_lag_12'] = avg_qty
        
        # Macro features (use last available values)
        if df_ip is not None and 'ip_value' in FEATURE_COLS:
            last_ip = df_ip[df_ip['date'] <= next_month].iloc[-1] if len(df_ip[df_ip['date'] <= next_month]) > 0 else df_ip.iloc[-1]
            row['ip_value'] = last_ip['ip_value']
            row['ip_change'] = 0  # Assume no change for future
            row['ip_pct_change'] = 0
            
        if df_msi is not None and 'msi_value' in FEATURE_COLS:
            last_msi = df_msi[df_msi['date'] <= next_month].iloc[-1] if len(df_msi[df_msi['date'] <= next_month]) > 0 else df_msi.iloc[-1]
            row['msi_value'] = last_msi['msi_value']
            row['msi_change'] = 0
            row['msi_pct_change'] = 0
        
        # Make prediction
        X_future = pd.DataFrame([row])[FEATURE_COLS]
        pred = best_model.predict(X_future)[0]
        
        row['revenue_forecast'] = pred
        future_predictions.append(row)
    
    return pd.DataFrame(future_predictions)

# Generate 12-month forecast
df_forecast = generate_future_features(df_clean, 12, df_ip, df_msi)
df_forecast_output = df_forecast[['month', 'revenue_forecast']].copy()
df_forecast_output['month'] = df_forecast_output['month'].dt.strftime('%Y-%m')

print("12-Month Revenue Forecast:")
display(df_forecast_output)

In [ ]:
# Save forecast results
df_forecast_output.to_csv('../data/interim/revenue_forecast_12m.csv', index=False)
print("✓ Forecast saved to data/interim/revenue_forecast_12m.csv")

In [ ]:
# Plot historical + forecast
plt.figure(figsize=(14, 6))

# Historical data
plt.plot(df_clean['month'], df_clean['revenue'], 'o-', 
         label='Historical (2016-2020)', color='#2E86AB', linewidth=2, markersize=4)

# Forecast
forecast_dates = pd.to_datetime(df_forecast['month'])
plt.plot(forecast_dates, df_forecast['revenue_forecast'], 's-', 
         label='Forecast (2021)', color='#E94F37', linewidth=2, markersize=6)

plt.xlabel('Month')
plt.ylabel('Revenue (₹)')
plt.title(f'AUM Monthly Revenue: Historical + 12-Month Forecast ({best_model_name})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('../reports/figures/revenue_forecast_12m.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Save the best model
joblib.dump(best_model, '../models/revenue_forecaster.pkl')
print(f"✓ Model saved to models/revenue_forecaster.pkl")

# Save feature list
with open('../models/feature_list.txt', 'w') as f:
    f.write('\n'.join(FEATURE_COLS))
print(f"✓ Feature list saved to models/feature_list.txt")

---
## Summary

### Model Selection
- Evaluated 6 models: Naive-1, Seasonal Naive-12, Moving Average (3), Moving Average (6), Random Forest, Gradient Boosting
- Selected model based on lowest MAPE on validation set (2019-2020)

### Features Used
- **Time-based**: year, month, quarter, is_year_end, is_year_start, is_quarter_end, month_sin, month_cos
- **Lag features**: revenue_lag_1, revenue_lag_3, revenue_lag_6, revenue_lag_12
- **Rolling features**: revenue_roll_mean_3, revenue_roll_mean_6, revenue_roll_mean_12, revenue_roll_std_3, revenue_roll_std_6
- **Macro features** (if available): ip_value, ip_change, ip_pct_change, msi_value, msi_change, msi_pct_change

### Results
- 12-month forecast generated and saved to `data/interim/revenue_forecast_12m.csv`
- Trained model saved to `models/revenue_forecaster.pkl`